In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests11 as vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:
def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [4]:
num_sims = 500
trials =1000
adapt_c = False
data_tuned_epsilon = False
epsilon = .5

size weirdness rn...

In [5]:
a = 0.25
num_params=9

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.96 & 0.96 & 0.95 & 0.84 & 0.95 & 0.97   \\
Model 1 & 0.00 & 0.00 & 0.01 & 0.07 & 0.01 & 0.01   \\
Model 2 & 0.03 & 0.03 & 0.04 & 0.09 & 0.04 & 0.01   \\
\hline
\end{tabular}
(array([0.964, 0.004, 0.032]), array([0.964, 0.004, 0.032]), array([0.954, 0.01 , 0.036]), array([0.84 , 0.066, 0.094]), array([0.954, 0.01 , 0.036]), array([0.974, 0.014, 0.012]), -3.4000709535317877, 10.377643245544016, 11.005997834999022)


# power 0.... need to see...

In [6]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9


def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [7]:
calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/5) )

nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.81 & 0.82 & 0.86 & 0.73 & 0.84 & 0.54   \\
Model 1 & 0.19 & 0.18 & 0.14 & 0.25 & 0.15 & 0.46   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.02 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.814, 0.186, 0.   ]), array([0.824, 0.176, 0.   ]), array([0.86 , 0.136, 0.004]), array([0.726, 0.254, 0.02 ]), array([0.842, 0.154, 0.004]), array([0.538, 0.462, 0.   ]), 6.712706064039718, 5.048148092879056, 5.358821006006797)


In [8]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.81 & 0.81 & 0.86 & 0.70 & 0.84 & 0.55   \\
Model 1 & 0.19 & 0.19 & 0.14 & 0.26 & 0.16 & 0.45   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.04 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.808, 0.192, 0.   ]), array([0.814, 0.186, 0.   ]), array([0.856, 0.14 , 0.004]), array([0.698, 0.264, 0.038]), array([0.842, 0.156, 0.002]), array([0.55, 0.45, 0.  ]), 6.849368038122052, 5.180405287925134, 5.337818102077535)


In [9]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.84 & 0.84 & 0.85 & 0.72 & 0.84 & 0.55   \\
Model 1 & 0.16 & 0.16 & 0.15 & 0.26 & 0.16 & 0.45   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.02 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.838, 0.162, 0.   ]), array([0.84, 0.16, 0.  ]), array([0.85, 0.15, 0.  ]), array([0.72 , 0.258, 0.022]), array([0.838, 0.162, 0.   ]), array([0.548, 0.452, 0.   ]), 6.5357768599822235, 5.013051253533495, 5.318967449588791)


In [10]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.80 & 0.80 & 0.86 & 0.73 & 0.85 & 0.50   \\
Model 1 & 0.20 & 0.20 & 0.14 & 0.24 & 0.15 & 0.50   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.02 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.802, 0.198, 0.   ]), array([0.804, 0.196, 0.   ]), array([0.864, 0.136, 0.   ]), array([0.734, 0.244, 0.022]), array([0.854, 0.146, 0.   ]), array([0.498, 0.502, 0.   ]), 6.976164481931269, 5.21406847353594, 5.332101305644023)


In [11]:
data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2)/50,nobs**(1/10) )

nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.81 & 0.81 & 0.86 & 0.70 & 0.85 & 0.56   \\
Model 1 & 0.19 & 0.19 & 0.14 & 0.27 & 0.15 & 0.44   \\
Model 2 & 0.00 & 0.00 & 0.01 & 0.03 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.808, 0.192, 0.   ]), array([0.812, 0.188, 0.   ]), array([0.858, 0.136, 0.006]), array([0.702, 0.266, 0.032]), array([0.848, 0.148, 0.004]), array([0.564, 0.436, 0.   ]), 6.622070316091068, 5.234304052251927, 5.346765498922524)


# a  = .25, k= 9

In [12]:
a = 0.25
num_params=9



In [13]:
nobs = 250

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.92 & 0.92 & 0.92 & 0.78 & 0.92 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.06 & 0.01 & 0.01   \\
Model 2 & 0.08 & 0.08 & 0.08 & 0.16 & 0.08 & 0.02   \\
\hline
\end{tabular}
(array([0.922, 0.   , 0.078]), array([0.924, 0.   , 0.076]), array([0.922, 0.002, 0.076]), array([0.784, 0.056, 0.16 ]), array([0.918, 0.006, 0.076]), array([0.978, 0.006, 0.016]), -3.8653087154868033, 5.590276909617487, 5.879089904229182)


In [14]:
nobs = 500

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.92 & 0.92 & 0.92 & 0.78 & 0.92 & 0.96   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.05 & 0.01 & 0.01   \\
Model 2 & 0.07 & 0.07 & 0.07 & 0.17 & 0.08 & 0.03   \\
\hline
\end{tabular}
(array([0.924, 0.008, 0.068]), array([0.924, 0.008, 0.068]), array([0.922, 0.006, 0.072]), array([0.776, 0.054, 0.17 ]), array([0.918, 0.006, 0.076]), array([0.96 , 0.014, 0.026]), -4.021904777151649, 8.152163978966847, 8.14789493445972)


In [15]:
nobs = 100

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.89 & 0.95 & 0.90 & 0.78 & 0.90 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.03 & 0.00 & 0.01   \\
Model 2 & 0.11 & 0.05 & 0.10 & 0.19 & 0.10 & 0.00   \\
\hline
\end{tabular}
(array([0.888, 0.   , 0.112]), array([0.952, 0.   , 0.048]), array([0.898, 0.   , 0.102]), array([0.778, 0.028, 0.194]), array([0.9  , 0.004, 0.096]), array([0.986, 0.01 , 0.004]), -4.246003018228189, 3.9971401877638026, 4.402202031133572)


# a  = .25, k= 4, something not right?

In [16]:
a = 0.25
num_params=4



In [17]:
nobs = 250

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.96 & 0.96 & 0.95 & 0.85 & 0.95 & 0.99   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.05 & 0.01 & 0.01   \\
Model 2 & 0.03 & 0.03 & 0.04 & 0.10 & 0.03 & 0.00   \\
\hline
\end{tabular}
(array([0.962, 0.008, 0.03 ]), array([0.962, 0.008, 0.03 ]), array([0.952, 0.01 , 0.038]), array([0.854, 0.046, 0.1  ]), array([0.954, 0.014, 0.032]), array([0.99 , 0.008, 0.002]), -1.3664936316056322, 5.31357947012954, 5.5943680246083565)


# a  = .25, K = 19

In [18]:
a = 0.25
num_params=19


In [19]:
nobs = 250

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.74 & 0.82 & 0.82 & 0.71 & 0.80 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.01   \\
Model 2 & 0.26 & 0.18 & 0.18 & 0.29 & 0.20 & 0.01   \\
\hline
\end{tabular}
(array([0.736, 0.   , 0.264]), array([0.822, 0.   , 0.178]), array([0.816, 0.   , 0.184]), array([0.708, 0.004, 0.288]), array([0.796, 0.   , 0.204]), array([0.978, 0.012, 0.01 ]), -9.382195636494135, 6.062758006082917, 6.706405115128255)


# evidence of power

In [20]:
nobs = 500

a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.98 & 0.97 & 0.78 & 0.97 & 0.65   \\
Model 1 & 0.02 & 0.02 & 0.02 & 0.12 & 0.02 & 0.35   \\
Model 2 & 0.01 & 0.00 & 0.01 & 0.10 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.974, 0.02 , 0.006]), array([0.98, 0.02, 0.  ]), array([0.972, 0.018, 0.01 ]), array([0.782, 0.116, 0.102]), array([0.97 , 0.016, 0.014]), array([0.654, 0.346, 0.   ]), 1.561263189197256, 5.449753518721526, 6.2398805274551545)


In [21]:
nobs = 100

a1,a2 = np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.97 & 1.00 & 0.96 & 0.91 & 0.95 & 0.67   \\
Model 1 & 0.02 & 0.00 & 0.02 & 0.06 & 0.03 & 0.33   \\
Model 2 & 0.01 & 0.00 & 0.02 & 0.04 & 0.02 & 0.00   \\
\hline
\end{tabular}
(array([0.966, 0.02 , 0.014]), array([0.998, 0.   , 0.002]), array([0.964, 0.02 , 0.016]), array([0.906, 0.058, 0.036]), array([0.948, 0.034, 0.018]), array([0.666, 0.334, 0.   ]), 0.7015022131758277, 5.597971206422079, 5.821411145026151)


In [22]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.55 & 0.55 & 0.72 & 0.59 & 0.72 & 0.51   \\
Model 1 & 0.45 & 0.45 & 0.28 & 0.39 & 0.28 & 0.49   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.02 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.55, 0.45, 0.  ]), array([0.55, 0.45, 0.  ]), array([0.716, 0.284, 0.   ]), array([0.59 , 0.394, 0.016]), array([0.718, 0.282, 0.   ]), array([0.508, 0.492, 0.   ]), 9.587474671644923, 4.859949266954477, 4.958298566153781)


In [23]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.98 & 0.96 & 0.83 & 0.95 & 0.67   \\
Model 1 & 0.02 & 0.01 & 0.02 & 0.09 & 0.04 & 0.33   \\
Model 2 & 0.01 & 0.00 & 0.02 & 0.08 & 0.02 & 0.00   \\
\hline
\end{tabular}
(array([0.97 , 0.022, 0.008]), array([0.984, 0.014, 0.002]), array([0.958, 0.024, 0.018]), array([0.826, 0.094, 0.08 ]), array([0.946, 0.036, 0.018]), array([0.668, 0.332, 0.   ]), 1.140897144105283, 5.628927537689673, 6.169843478337177)


# evidence of power 2

In [24]:
nobs = 250


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.02 & 0.55 & 0.22 & 0.30 & 0.23 & 0.65   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.98 & 0.45 & 0.78 & 0.70 & 0.77 & 0.35   \\
\hline
\end{tabular}
(array([0.022, 0.   , 0.978]), array([0.554, 0.   , 0.446]), array([0.22, 0.  , 0.78]), array([0.296, 0.002, 0.702]), array([0.234, 0.   , 0.766]), array([0.648, 0.   , 0.352]), -20.04355252758754, 5.806012741275091, 6.107575580044739)


In [25]:
nobs = 500


a2,a1 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.01 & 0.35 & 0.34 & 0.42 & 0.36 & 0.67   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.99 & 0.65 & 0.66 & 0.58 & 0.64 & 0.33   \\
\hline
\end{tabular}
(array([0.01, 0.  , 0.99]), array([0.35, 0.  , 0.65]), array([0.34, 0.  , 0.66]), array([0.42, 0.  , 0.58]), array([0.36, 0.  , 0.64]), array([0.666, 0.   , 0.334]), -20.317620601073695, 5.887114074280808, 6.292490839080534)


In [26]:
nobs = 100


a2,a1 =np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.01 & 0.93 & 0.03 & 0.11 & 0.06 & 0.54   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.99 & 0.07 & 0.97 & 0.89 & 0.94 & 0.46   \\
\hline
\end{tabular}
(array([0.006, 0.   , 0.994]), array([0.932, 0.   , 0.068]), array([0.034, 0.   , 0.966]), array([0.106, 0.   , 0.894]), array([0.058, 0.   , 0.942]), array([0.54, 0.  , 0.46]), -21.090882617928308, 5.296335712558832, 5.796206962932405)


In [27]:
nobs = 250


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.21 & 0.22 & 0.53 & 0.49 & 0.52 & 0.53   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
Model 2 & 0.79 & 0.78 & 0.47 & 0.49 & 0.48 & 0.47   \\
\hline
\end{tabular}
(array([0.21, 0.  , 0.79]), array([0.218, 0.   , 0.782]), array([0.532, 0.   , 0.468]), array([0.492, 0.014, 0.494]), array([0.524, 0.   , 0.476]), array([0.532, 0.   , 0.468]), -12.28367169061438, 4.91745708003817, 4.901763286218437)


In [28]:
nobs = 250


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.02 & 0.56 & 0.21 & 0.26 & 0.20 & 0.63   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.98 & 0.44 & 0.79 & 0.74 & 0.80 & 0.37   \\
\hline
\end{tabular}
(array([0.024, 0.   , 0.976]), array([0.556, 0.   , 0.444]), array([0.208, 0.   , 0.792]), array([0.26, 0.  , 0.74]), array([0.196, 0.   , 0.804]), array([0.632, 0.   , 0.368]), -20.30676375282023, 6.029274151134176, 6.176983867465149)
